In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classificationcopy/data_inlf_train.csv
/kaggle/input/classificationcopy/data_inlf_test.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import metrics
import sys

def main():
    train = pd.read_csv("/kaggle/input/classificationcopy/data_inlf_train.csv")
    test = pd.read_csv("/kaggle/input/classificationcopy/data_inlf_test.csv")

    X = train.drop(['inlf'], axis=1)
    y = train['inlf']

    x_train, x_test, y_train, y_test = trainTest(X, y)
    logisticAccuracy = logisticRegressionModel(x_train, x_test, y_train, y_test)
    dectreeAccuracy = decisionTreeModel(x_train, x_test, y_train, y_test)
    svmAccuracy = svmModel(x_train, x_test, y_train, y_test)
    printComparisonResults(logisticAccuracy,dectreeAccuracy,svmAccuracy)


def trainTest(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=0, stratify=y)
    return x_train, x_test, y_train, y_test

# LogisticRegression model
def logisticRegressionModel(x_train, x_test, y_train, y_test):
    logistic_reg = LogisticRegression().fit(x_train, y_train)
    y_pred = logistic_reg.predict(x_test)

    #evaluate accuracy of our model
    acc = metrics.accuracy_score(y_test, y_pred)

    return acc
def decisionTreeModel(x_train, x_test, y_train, y_test):
    dec_tree = DecisionTreeClassifier()
    dec_tree = dec_tree.fit(x_train, y_train)
    y_pred = dec_tree.predict(x_test)

    #evaluate model
    acc = metrics.accuracy_score(y_test, y_pred)

    return acc
def svmModel(x_train, x_test, y_train, y_test):
    svm_model = svm.SVC(kernel='linear')
    svm_model = svm_model.fit(x_train, y_train)
    y_pred = svm_model.predict(x_test)

    #evaluate model
    acc = metrics.accuracy_score(y_test, y_pred)

    return acc
def printComparisonResults(logisticAccuracy,dectreeAccuracy,svmAccuracy):
    print("Printing Results")

    lr = "Logistic Regression"
    dt = "Decision Tree"
    sv = "SVM is Accurate"

    if logisticAccuracy > (dectreeAccuracy and svmAccuracy):
        print(lr, "is accurate than", dt, "and", sv)
    elif dectreeAccuracy > (logisticAccuracy and svmAccuracy):
        print(dt, "is accurate than", lr, "and", sv)
    else:
        print(sv, "is accurate than", lr, "and", dt)

    print("Done")
if __name__ == '__main__': 
    main()

Printing Results
SVM is Accurate is accurate than Logistic Regression and Decision Tree
Done


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import numpy as np

train = pd.read_csv("/kaggle/input/classificationcopy/data_inlf_train.csv")
test = pd.read_csv("/kaggle/input/classificationcopy/data_inlf_test.csv")


X = train.drop(['inlf'], axis=1)
y = train['inlf']

# create a linear regression model
svm = svm.SVC(kernel='linear')

# generate some training data and targets
X_train = X
y_train = y

# set up a KFold object with 5 folds
kf = KFold(n_splits=5)

# initialize an array to hold the cross-validation scores
scores = []

# loop over the folds
for train_index, test_index in kf.split(X_train):
    # split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit the model to the training data
    svm.fit(X_train, y_train)
    predictions = svm.predict(X_test)
    predictions_train = svm.predict(X_train) 
    
    # evaluate the model on the testing data and store the score
    test_acc = metrics.accuracy_score(y_test, predictions)
    train_acc = metrics.accuracy_score(y_train, predictions_train)
    scores.append((train_acc, test_acc))

# print the mean and standard deviation of the cross-validation scores
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.2f}".format(np.mean(scores)))
print("Standard deviation of cross-validation scores: {:.2f}".format(np.std(scores)))

Cross-validation scores: [(0.6274944567627494, 0.6460176991150443), (0.6452328159645233, 0.6283185840707964), (0.6629711751662971, 0.5575221238938053), (0.6452328159645233, 0.6637168141592921), (0.6438053097345132, 0.6339285714285714)]
Average cross-validation score: 0.64
Standard deviation of cross-validation scores: 0.03


In [6]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn import svm

# Define the hyperparameter space
hyperparam_space = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

X_train = X
y_train = y

# Create the random forest regressor object
svc = svm.SVC()

# Create the randomized search CV object
random_search = RandomizedSearchCV(svc, hyperparam_space, n_iter=50, cv=5)

# Fit the randomized search CV object
random_search.fit(X_train, y_train)

# Evaluate the best model on the test data
best_model = random_search.best_estimator_
test_error = best_model.score(X_test, y_test)

print(best_model)
print(test_error)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 25 is smaller than n_iter=50. Running 25 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


SVC(C=1000, gamma=0.0001)
0.7142857142857143


In [7]:
submissions = best_model.predict(test)

In [8]:
submission =pd.DataFrame(np.array(submissions), columns=['Expected'])
submission.head()

,Expected
0,1
1,1
2,0
3,1
4,0


In [9]:
submission['id'] = submission.index
submission.head()

,Expected,id
0,1,0
1,1,1
2,0,2
3,1,3
4,0,4


In [10]:
submission[['id', 'Expected']].to_csv('submission_clf.csv', index=False)